# RoBERTa ile Veri Setinden Öznitelik Çıkarımı

Öncelikle MELD veri setini yükleyip tokenize ederek işleme hazır hale getirdim.

In [1]:
import pandas as pd

# MELD veri setini yükle
meld_dev = pd.read_csv("../../data/raw/dev_sent_emo.csv")

# İlgili sütunları al
meld_texts = meld_dev["Utterance"].tolist()
meld_labels = meld_dev["Emotion"].tolist()
meld_speaker = meld_dev["Speaker"].tolist()
meld_conversation = meld_dev["Dialogue_ID"].tolist()

RoBERTa tokenizer ile metinleri işledim.

In [2]:
from transformers import RobertaTokenizer

# Tokenizer'ı yükle
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

# Bütün metinleri tokenize et
tokens = tokenizer(
    meld_texts, 
    padding="max_length",  # Maksimum uzunluğa göre pad ekler
    truncation=True,       # Uzun metinleri keser
    max_length=128,        # Maksimum uzunluk
    return_tensors="pt"    # PyTorch tensörü olarak döndürür
)

print(tokens["input_ids"].shape)  # (num_samples, 128)
print(tokens)


/Users/sudetungac/miniconda3/envs/bitirme-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1109, 128])
{'input_ids': tensor([[    0,  7516,   127,  ...,     1,     1,     1],
        [    0,  2264,   116,  ...,     1,     1,     1],
        [    0, 11094,   328,  ...,     1,     1,     1],
        ...,
        [    0,   100,  4056,  ...,     1,     1,     1],
        [    0,   100,  1266,  ...,     1,     1,     1],
        [    0,   100,   216,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


Hugging Face kütüphanesini kullanarak RoBERTa gömme özniteliklerini çıkardım.

In [7]:
from transformers import RobertaModel
import torch

# RoBERTa modelini yükle
model = RobertaModel.from_pretrained("roberta-large")

# Model ile embedding çıkartma
with torch.no_grad():
    outputs = model(**tokens)

# Son gizli katman çıktısı (CLS token embedding'i kullan)
roberta_features = outputs.last_hidden_state[:, 0, :]  # (batch_size, hidden_dim)

# Özellik boyutu
print(roberta_features.shape)  # (num_samples, 768)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1109, 1024])


DialogueRNN modeli, konuşmaları sıralı olarak işlediği için veriyi konuşmalara göre gruplayıp işlemek gerekiyor.
Her konuşmanın diyalog sırasını koruyarak bir liste formatına çevirdim.

In [8]:
from collections import defaultdict

# Diyalogları gruplama
dialogues = defaultdict(list)
for text, speaker, conv_id, feature, label in zip(meld_texts, meld_speaker, meld_conversation, roberta_features, meld_labels):
    dialogues[conv_id].append((feature, speaker, label))

# Sıralı diyalog listesi oluştur
sorted_dialogues = [dialogues[key] for key in sorted(dialogues.keys())]
